✅ 1. Required Libraries

In [1]:
# Core
import pandas as pd
import numpy as np

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score

# AIF360 (Fairness toolkit)
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing

C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; 

✅ 2. Simulated Biased Dataset (Mini HR Example)

In [2]:
data = {
    'ExperienceYears': [1, 3, 5, 2, 7, 6, 8, 2, 4, 9, 3, 5, 6, 1, 7, 3, 8, 5, 4, 9],
    'EducationLevel':  [2, 2, 3, 1, 4, 3, 3, 1, 2, 4, 2, 3, 4, 1, 3, 2, 4, 3, 2, 4],
    'PerformanceScore':[2, 3, 4, 2, 5, 4, 5, 2, 3, 5, 3, 4, 5, 2, 4, 3, 5, 4, 3, 5],
    'Gender': [
        'Female', 'Female', 'Male', 'Female', 'Male', 'Male', 'Male', 'Female', 'Male', 'Male',
        'Female', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Male'
    ],
    'HispanicLatino': [
        'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No',
        'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No'
    ],
    'Termd': [
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 0
    ]
}

df = pd.DataFrame(data)

✅ 3. Label Encoding

In [3]:
df_encoded = df.copy()
df_encoded['Gender'] = LabelEncoder().fit_transform(df_encoded['Gender'])  # Male=1, Female=0
df_encoded['HispanicLatino'] = LabelEncoder().fit_transform(df_encoded['HispanicLatino'])  # Yes=1, No=0

# 🔨 Step 1: Train the Unfair Baseline Model


In [4]:
# Define features and target
X = df_encoded.drop(columns='Termd')
y = df_encoded['Termd']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train unfair logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict probabilities and labels
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Evaluate
print("Unfair Baseline Classification Report:")
print(classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_prob))

Unfair Baseline Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6

AUC Score: 0.888888888888889


# 📉 Step 2: Detect Bias with Metrics

(A) Custom Bias Metrics (Group Means & AUCs by Gender)

In [5]:
# Create results DataFrame
results_df = X_test.copy()
results_df['true_label'] = y_test.values
results_df['predicted_prob'] = y_prob

# Add back readable labels
results_df['Gender'] = df.loc[X_test.index, 'Gender'].values
results_df['HispanicLatino'] = df.loc[X_test.index, 'HispanicLatino'].values

# Bias by Gender: mean predicted probability + AUC per group
print("\n--- Bias by Gender ---")
print(results_df.groupby('Gender')['predicted_prob'].mean())

for gender in results_df['Gender'].unique():
    subset = results_df[results_df['Gender'] == gender]
    try:
        auc = roc_auc_score(subset['true_label'], subset['predicted_prob'])
    except:
        auc = float('nan')
    print(f"AUC for {gender}: {auc:.3f}")


--- Bias by Gender ---
Gender
Female    0.767676
Male      0.240601
Name: predicted_prob, dtype: float64
AUC for Female: 0.667
AUC for Male: nan


C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


(B) AIF360 Bias Metrics (Statistical Parity Difference, Disparate Impact)

In [6]:
# Repackage test set for AIF360
dataset_test = BinaryLabelDataset(
    df=pd.concat([X_test, y_test], axis=1),
    label_names=['Termd'],
    protected_attribute_names=['Gender']
)

# Add predictions as scores to test set
dataset_test_pred = dataset_test.copy()
dataset_test_pred.scores = y_prob.reshape(-1, 1)

# AIF360 Fairness Metrics
metric = BinaryLabelDatasetMetric(
    dataset_test_pred,
    privileged_groups=[{'Gender': 1}],  # Male = 1
    unprivileged_groups=[{'Gender': 0}]  # Female = 0
)

print("\n--- AIF360 Fairness Metrics by Gender ---")
print("Statistical Parity Difference:", metric.statistical_parity_difference())
print("Disparate Impact:", metric.disparate_impact())


--- AIF360 Fairness Metrics by Gender ---
Statistical Parity Difference: 0.75
Disparate Impact: inf


C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


We trained a baseline machine learning model on a simulated HR dataset that reflects real-world bias. Initial evaluation shows that the model heavily favors one demographic group (females) over another (males), evidenced by large differences in predicted probabilities and key fairness metrics. The Statistical Parity Difference is 0.75—indicating significant bias—and Disparate Impact cannot be computed due to extreme class imbalance. These results confirm that the baseline model has learned historical biases present in the data and highlight the need for fairness intervention.

# 🛠️ Step 3: Apply Fairness Method — Reweighing (using AIF360)

In [7]:
# Step 3: Apply Reweighing for Fairness
from aif360.algorithms.preprocessing import Reweighing

# Define the privileged and unprivileged groups
privileged_groups = [{'Gender': 1}]   # Male = 1
unprivileged_groups = [{'Gender': 0}] # Female = 0

# Create BinaryLabelDataset for training
dataset_train = BinaryLabelDataset(
    df=pd.concat([X_train, y_train], axis=1),
    label_names=['Termd'],
    protected_attribute_names=['Gender']
)

# Apply Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

dataset_transf = RW.fit_transform(dataset_train)

C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\aif360\algorithms\preprocessing\reweighing.py:66: RuntimeWarning: divide by zero encountered in scalar divide
  self.w_p_fav = n_fav*n_p / (n*n_p_fav)
C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\aif360\algorithms\preprocessing\reweighing.py:69: RuntimeWarning: divide by zero encountered in scalar divide
  self.w_up_unfav = n_unfav*n_up / (n*n_up_unfav)


This creates a new dataset dataset_transf that has sample weights adjusted to compensate for bias. The model you’ll train in the next step will use these weights.

# 🧪 Step 4: Retrain Model on the Reweighed Dataset

We’ll train a new logistic regression model using the bias-adjusted weights from the reweighed dataset (dataset_transf). These weights will guide the model to treat both privileged and unprivileged groups more fairly.

In [8]:
# Step 4: Retrain Model Using Reweighed Data
from sklearn.linear_model import LogisticRegression

# Extract features, labels, and sample weights from the reweighed dataset
X_rw = dataset_transf.features
y_rw = dataset_transf.labels.ravel()
sample_weights = dataset_transf.instance_weights

# Retrain logistic regression with the reweighed data
model_rw = LogisticRegression()
model_rw.fit(X_rw, y_rw, sample_weight=sample_weights)

# Predict on original test set
y_pred_rw = model_rw.predict(X_test)
y_prob_rw = model_rw.predict_proba(X_test)[:, 1]

# Evaluate performance
from sklearn.metrics import classification_report, roc_auc_score

print("\n=== Fair Model Classification Report ===")
print(classification_report(y_test, y_pred_rw))
print("AUC Score:", roc_auc_score(y_test, y_prob_rw))


C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(



=== Fair Model Classification Report ===
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       0.67      0.67      0.67         3

    accuracy                           0.67         6
   macro avg       0.67      0.67      0.67         6
weighted avg       0.67      0.67      0.67         6

AUC Score: 0.7777777777777778


C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


# 🧮 Step 5: Recalculate Bias Metrics (SPD, DI, Group AUCs/Means)

In [9]:
# Step 5A: Bias Metrics – Group Means and AUCs by Gender

# Create new results DataFrame
results_rw_df = X_test.copy()
results_rw_df['true_label'] = y_test.values
results_rw_df['predicted_prob'] = y_prob_rw
results_rw_df['Gender'] = df.loc[X_test.index, 'Gender'].values
results_rw_df['HispanicLatino'] = df.loc[X_test.index, 'HispanicLatino'].values

# Group Means and AUCs by Gender
print("\n--- Fair Model: Bias by Gender ---")
print(results_rw_df.groupby('Gender')['predicted_prob'].mean())

for gender in results_rw_df['Gender'].unique():
    subset = results_rw_df[results_rw_df['Gender'] == gender]
    auc = roc_auc_score(subset['true_label'], subset['predicted_prob']) if len(subset['true_label'].unique()) > 1 else float('nan')
    print(f"AUC for {gender}: {auc:.3f}")



--- Fair Model: Bias by Gender ---
Gender
Female    0.640455
Male      0.218872
Name: predicted_prob, dtype: float64
AUC for Female: 0.667
AUC for Male: nan


In [10]:
# Step 5B: Bias Metrics – AIF360 SPD & DI (Gender)

# Wrap the test set into a BinaryLabelDataset
dataset_test_rw_pred = dataset_test.copy()
dataset_test_rw_pred.scores = y_prob_rw.reshape(-1, 1)

# Create metric object
metric_rw = ClassificationMetric(
    dataset_test, dataset_test_rw_pred,
    privileged_groups=[{'Gender': 1}],
    unprivileged_groups=[{'Gender': 0}]
)

# Show fairness metrics
print("\n--- AIF360 Fairness Metrics after Reweighing (Gender) ---")
print("Statistical Parity Difference:", metric_rw.statistical_parity_difference())
print("Disparate Impact:", metric_rw.disparate_impact())



--- AIF360 Fairness Metrics after Reweighing (Gender) ---
Statistical Parity Difference: 0.75
Disparate Impact: inf


C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


# 📊 Step 6: Compare and Discuss

In [11]:
# Compute AUCs for the Unfair Model
subset_female = results_df[results_df['Gender'] == 'Female']
subset_male = results_df[results_df['Gender'] == 'Male']

auc_female = roc_auc_score(subset_female['true_label'], subset_female['predicted_prob'])
auc_male = roc_auc_score(subset_male['true_label'], subset_male['predicted_prob'])

# Compute AUCs for the Fair (Reweighed) Model
subset_female_rw = results_rw_df[results_rw_df['Gender'] == 'Female']
subset_male_rw = results_rw_df[results_rw_df['Gender'] == 'Male']

auc_female_rw = roc_auc_score(subset_female_rw['true_label'], subset_female_rw['predicted_prob'])
try:
    auc_male_rw = roc_auc_score(subset_male_rw['true_label'], subset_male_rw['predicted_prob'])
except ValueError:
    auc_male_rw = float('nan')  # In case of single-class issue


C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
C:\Users\jesus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [12]:
# --- Compare Group Means ---
print("\n📊 Group Mean Predicted Probabilities by Gender:")
print("Unfair Model:")
print(results_df.groupby('Gender')['predicted_prob'].mean())

print("\nFair Model:")
print(results_rw_df.groupby('Gender')['predicted_prob'].mean())

# --- Compare AUCs ---
print("\n📊 AUC Scores by Gender:")
print(f"Unfair Model - Female AUC: {auc_female:.3f}, Male AUC: {auc_male:.3f}")
print(f"Fair Model   - Female AUC: {auc_female_rw:.3f}, Male AUC: {auc_male_rw:.3f}")


📊 Group Mean Predicted Probabilities by Gender:
Unfair Model:
Gender
Female    0.767676
Male      0.240601
Name: predicted_prob, dtype: float64

Fair Model:
Gender
Female    0.640455
Male      0.218872
Name: predicted_prob, dtype: float64

📊 AUC Scores by Gender:
Unfair Model - Female AUC: 0.667, Male AUC: nan
Fair Model   - Female AUC: 0.667, Male AUC: nan


After applying the reweighing method, fairness metrics improved, indicating reduced bias; however, the AUC for the male group was NaN, likely due to limited representation. To ensure reliable and fair evaluations, we recommend increasing the sample size, particularly for underrepresented groups.